In [76]:
from elasticsearch import Elasticsearch
import re
import os
import pandas as pd
import string
from elasticsearch_dsl import Search
import json
import math

In [77]:
es = Elasticsearch("http://localhost:9200", timeout=60)

In [78]:
os.chdir("/Users/Dibble/Desktop/homework-1-Evan-Chan-NEU-main/IR_data/AP_DATA")
files = os.listdir("/Users/Dibble/Desktop/homework-1-Evan-Chan-NEU-main/IR_data/AP_DATA/ap89_collection")
files.remove("readme")

In [79]:
# function (also used in parser/indexer) to get list of docnums
def get_docnums():
    test_keys = []
    for i in files[:364]:
        current = open(str("/Users/Dibble/Desktop/homework-1-Evan-Chan-NEU-main/IR_data/AP_DATA/ap89_collection/"+i), encoding = "latin-1")
        text = current.read()
        keys = re.findall("<DOCNO>(.*)</DOCNO>", text)
        for j in keys:
            test_keys.append(j)
    return test_keys

In [80]:
def find_anom(text):
    between = re.findall("</TEXT>(.*?)<TEXT>", text)
    anoms = [x for x in between if "<DOCNO>" not in x]
    for n in anoms:
        if n in text:
            text = text.replace(n, "")
    return text

In [81]:
def remove_others(text):
    fileid = re.findall("<FILEID>(.+?)</FILEID>", text)
    first = re.findall("<FIRST>(.+?)</FIRST>", text)
    second = re.findall("<SECOND>(.+?)</SECOND>", text)
    head = re.findall("<HEAD>(.+?)</HEAD>", text)
    byline = re.findall("<BYLINE>(.+?)</BYLINE>", text)
    dateline = re.findall("<DATELINE>(.+?)</DATELINE>", text)
    note = re.findall("<NOTE>(.+?)</NOTE>", text)
    removal = fileid + first + second + head + byline + dateline + note
    for x in removal:
        if x in text:
            text = text.replace(x, "")
    sections = ["<DOC>", "</DOC>", "<FILEID>", "</FILEID>", "<FIRST>", "</FIRST>", "<SECOND>", "</SECOND>", 
    "<HEAD>", "</HEAD>", "<BYLINE>", "</BYLINE>", "<DATELINE>", "</DATELINE>", "<NOTE>", "</NOTE>", 
    "</TEXT><TEXT>", "</TEXT> <TEXT>", "</TEXT>  <TEXT>", "</TEXT>   <TEXT>", 
    "</TEXT> files a0511-a0514-a0517-a0518; inserts new 5thgraf, In another, with leading indicators report; edits pvs 5th graf fortransition<TEXT>", 
    "</TEXT><UNK>Eds: For release 11 a.m. EDT, time set by source.</UNK><TEXT>", "</TEXT> For release 8:01 p.m. EDT<TEXT>", 
    "</TEXT> ^Consumer Advocates, Insurers Join Forces<TEXT>", "</TEXT>By s<TEXT>", "</TEXT>s<TEXT>"]
    for y in sections:
        if y in text:
            text = text.replace(y, "")
    return text

In [82]:
def get_texts():
    test_values = []
    for i in files[:364]:
        current = open(str("/Users/Dibble/Desktop/homework-1-Evan-Chan-NEU-main/IR_data/AP_DATA/ap89_collection/"+i), encoding = 'latin-1')
        text = current.read()
        text = ''.join(text.splitlines())
        text = find_anom(text)
        text = remove_others(text)
        values = re.findall("<TEXT>(.*?)</TEXT>", text)
        for j in values:
            test_values.append(j)
    return test_values

In [83]:
def get_sw():
    with open("/Users/Dibble/Desktop/homework-1-Evan-Chan-NEU-main/IR_data/AP_DATA/stoplist.txt") as sw:
        stop_words = sw.read().split()
    return stop_words

In [84]:
def format_text():
    formatted_values = []
    stop_words = get_sw()
    test_values = get_texts()

    for text in test_values:
        new_string = re.sub("-", " ", text)
        new_string = re.sub("\n", " ", new_string)
        new_string = new_string.translate(str.maketrans('', '', string.punctuation))
        new_string = ' '.join([word for word in new_string.split() if word not in stop_words])
        formatted_values.append(new_string.lower())
    return formatted_values

#formatted_values = format_text()
#print(formatted_values[1])

In [85]:
# function to get vocabulary size of corpus - unique terms in collection
def get_vocab_size():
    formatted_values = format_text()
    one_string = " ".join(formatted_values)
    all_words = one_string.split()
    unique_words = set(all_words)
    vocab = len(unique_words)
    return vocab

#get_vocab_size()

In [86]:
# helper function for get_queries() to return the number of each query
def get_queries_nums(queries_list):
    queries_nums = []
    for line in queries_list:
        num = line.split()[0]
        queries_nums.append(num)
    return queries_nums

In [87]:
# helper function for get_queries() to return necessary text of each query with
def get_queries_text(queries_list):
    queries_nums = get_queries_nums(queries_list)
    query_stop_words = ["Document", "will", "must", "discuss", 
    "report", "include", "describe", "identify", "a", "an", "as",
    "and", "the", "to", "or", "either", "of", "by", "in", "with", 
    "about", "some", "any", "its", "even", "other", "which",
    "being", "certain", "has"]
    remove_words = queries_nums + query_stop_words
    queries_text = []
    for i in queries_list:
        x = i.split()
        queries_text.append(" ".join(a if a not in remove_words else '' for a in x))
    return queries_text

In [88]:
# function to get dataframe with query number and query text
def get_queries_df():
    queries_df = pd.DataFrame()
    with open("/Users/Dibble/Desktop/homework-1-Evan-Chan-NEU-main/IR_data/AP_DATA/query_desc.51-100.short.txt") as queries_file:
        queries_list = queries_file.read()
        queries_list = re.sub("-", " ", queries_list)
        queries_list = queries_list.translate(str.maketrans('', '', string.punctuation))
        queries_list = queries_list.split("\n")
        queries_nums = get_queries_nums(queries_list)
        queries_text = get_queries_text(queries_list)
        queries_df["QueryNumber"] = queries_nums
        queries_df["QueryText"] = queries_text
    return queries_df

#queries_df = get_queries_df()
#print(queries_df)

In [89]:
# function to get list of queries 
def get_queries_words(queries_df):
    query_words_list = []
    for query in queries_df.iloc[:, 1]:
        query_words_list.append(query)
    return query_words_list

#queries_df = get_queries_df()
#query_words_list = get_queries_words(queries_df)
#print(query_words_list)

In [90]:
# helper function to search term vector api result for query word and return its frequency within a document - how many times it is in document
def get_tf_num(term, results):
    tf_num = 0
    if results.__contains__("TEXT") is True:
        if term in results["TEXT"]["terms"]:
            tf = results["TEXT"]["terms"][term]["term_freq"]
            tf_num += tf
            return tf_num
        else:
            return tf_num
    else:
        return tf_num

In [91]:
# helper function to search term vector api result for query word and return its document frequency - how many documents it is in
def get_df_num(term, results):
    df_num = 0
    if results.__contains__("TEXT") is True:
        if term in results["TEXT"]["terms"]:
            df = results["TEXT"]["terms"][term]["doc_freq"]
            df_num += df
            return df_num
        else:
            return df_num
    else:
        return df_num

In [92]:
# helper function to get term frequency of query word within the query - how many times is it in the query
def get_tq_num(term, query):
    tq_num = query.count(term)
    return tq_num

In [93]:
# helper function to retrieve dictionary of query words tf within a document
def get_docnum_tf_dict(docnum, query):
    q_word_dict = {}
    results = es.termvectors(index="ap89_index4",
                        id=str(docnum),
                        body={
                            "fields": ["TEXT"],
                            "term_statistics": True,
                            "field_statistics": True
                        })["term_vectors"]
    for term in query:
        q_word_dict[term] = [get_tf_num(term, results), get_df_num(term, results), get_tq_num(term, query)]
    
    return q_word_dict

In [94]:
# function to retrieve term frequency values and document frequency values of words in all 25 queries in each of the 84678 documents
# returns dictionary of {docnum : {q_word : [tf, df, tq], q_word : [tf, df, tq]}, docnum : {q_word : [tf, df, tq], q_word : [tf, df, tq]}, etc.}
def create_tf_df_tq_dict():
    queries_df = get_queries_df()
    q_nums_list = queries_df.iloc[:, 0]
    q_words_list = queries_df.iloc[:, 1]
    test_keys = get_docnums()

    tf_df_tq_dict_list = []
    for list in q_words_list[:25]:
        query = list.split()
        tf_dict = {}
        for docnum in test_keys[:84678]:
            tf_dict[docnum] = get_docnum_tf_dict(docnum, query)
        tf_df_tq_dict_list.append(tf_dict)
    
    return tf_df_tq_dict_list

#tf_df_tq_dict_list = create_tf_df_tq_dict()
#print(tf_df_tq_dict_list)

In [102]:
# helper function to get the doc length of a given doc
def get_doc_length(doc):
    doc_length = 0
   
    results = es.termvectors(index="ap89_index4",
                        id=str(doc),
                        body={
                            "fields": ["TEXT"],
                            "term_statistics": True,
                            "field_statistics": True
                        })["term_vectors"]

    if results.__contains__("TEXT") is True:
        for term in results["TEXT"]["terms"]:
            tf_val = 1 * (results["TEXT"]["terms"][term]["term_freq"])
            doc_length += tf_val
        return doc_length
    else:
        return doc_length

In [96]:
# helper function to get dataframe of uni lap scores for 1 query for all documents
def get_uni_lap_df(tf_df_tq_dict, count):
 
   doc_length_list = []
   for doc in tf_df_tq_dict.keys():
      doc_length = get_doc_length(doc)
      doc_length_list.append(doc_length)
   
   doc_id_list = []
   for doc in tf_df_tq_dict.keys():
      doc_id_list.append(doc)

   test_keys = get_docnums()
   corpus_size = len(test_keys)
   avg_doc_length = (sum(doc_length_list))/corpus_size

   queries_df = get_queries_df()
   q_n_list = queries_df["QueryNumber"].tolist()

   uni_lap_score = []
   q_tf_df_tq_list = [i for i in tf_df_tq_dict.values()]
   vocab = get_vocab_size()
   for index, list in enumerate(q_tf_df_tq_list):
      current_list = []
      uni_lap = 0
      for tf_df_tq in list.values():
         doc_length = doc_length_list[index]
         
         numer = (tf_df_tq[0] + 1)
         denom = (doc_length + vocab)
         
         if tf_df_tq[0] == 0:
            uni_lap += -1000
         else:
            uni_lap += math.log(numer/denom)
         
         current_list.append(uni_lap)

      uni_lap_score.append(sum(current_list))
   
   uni_lap_df = pd.DataFrame()
   uni_lap_df = uni_lap_df.reset_index()
   uni_lap_df["QUERYNUM"] = [q_n_list[count]] * len(doc_id_list)
   uni_lap_df["DOCNO"] = doc_id_list
   uni_lap_df["SCORE"] = uni_lap_score
   result_df = uni_lap_df.sort_values(by="SCORE", ascending=False)
   result_df = result_df.iloc[:1000]

   return result_df

In [103]:
# main function to write retrieval model results to output file
def unigram_laplace():
    tf_df_tq_dict_list = create_tf_df_tq_dict()

    uni_lap_results_list = []
    count = 0
    for tf_df_tq_dict in tf_df_tq_dict_list:
        result_df = get_uni_lap_df(tf_df_tq_dict, count)
        count += 1
        uni_lap_results_list.append(result_df)
    
    for df in uni_lap_results_list:
        query_number_result = df["QUERYNUM"].tolist()
        query_result_docnums = df["DOCNO"].tolist()
        query_result_scores = df["SCORE"].tolist()
        with open('/Users/Dibble/Desktop/homework-1-Evan-Chan-NEU-main/Unigram_LM_Laplace_smoothing_results.txt', 'a') as queryResults:
            rank = 1
            j = 0
            while j in range(len(query_result_docnums)):
                queryResults.write('%s Q0 %s %s %s Exp\n' % (query_number_result[j], query_result_docnums[j], rank, query_result_scores[j]))
                rank += 1
                j += 1
        queryResults.close()

unigram_laplace()